# Drone Trajectory Planner

In this project, we will develop the drone trajectory planner. This notebook serves as the main file for the project, where we will refer to the instructions and demonstrate our code.

Please follow week by week instructions, which includes writing the code in the `src/` folder.

In [1]:
# Import all the files and libraries required for the project
%load_ext autoreload
%autoreload 2
import copy
    
import numpy as np

from src.camera_utils import compute_image_footprint_on_surface, compute_ground_sampling_distance, project_world_point_to_image
from src.data_model import Camera, DatasetSpec
from src.plan_computation import compute_distance_between_images, compute_speed_during_photo_capture, generate_photo_plan_on_grid
from src.visualization import plot_photo_plan

import plotly.io as pio
pio.renderers.default = 'iframe'

# Week 1: Introduction

No code contribution expected this week

# Week 2: Implement data models for dataset specification and camera

For this week, we will model the dataset specification.

- Overlap: the ratio (in 0 to 1) of scene shared between two consecutive images.
- Sidelap: the ratio (in 0 to 1) of scene shared between two images in adjacent rows.
- Height: the height of the scan above the ground (in meters).
- Scan_dimension_x: the horizontal size of the rectangle to be scanned (in meters).
- Scan_dimension_y: the vertical size of the rectangle to be scanned (in meters).
- exposure_time_ms: the exposure time for each image (in milliseconds).


$\color{red}{\text{TODO: }}$ Implement `DatasetSpec` in `src/data_model.py`


In [2]:
# Model the nomimal dataset spec

overlap = 0.7
sidelap = 0.7
height = 30.48 # 100 ft
scan_dimension_x = 150
scan_dimension_y = 150
exposure_time_ms = 2 # 1/500 exposure time

dataset_spec = DatasetSpec(overlap, sidelap, height, scan_dimension_x, scan_dimension_y, exposure_time_ms)

print(f"Nominal specs: {dataset_spec}")

Nominal specs: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48m, scan_area=150x150m, exposure=2ms)


## Model the camera parameters

We want to model the following camera parameters in Python:
- focal length along x axis (in pixels)
- focal length along y axis (in pixels)
- optical center of the image along the x axis (in pixels)
- optical center of the image along the y axis (in pixels)
- Size of the sensor along the x axis (in mm)
- Size of the sensor along the y axis (in mm)
- Number of pixels in the image along the x axis
- Number of pixels in the image along the y axis

I recommend to use `dataclasses` ([Python documentation](https://docs.python.org/3/library/dataclasses.html), [Blog](https://www.dataquest.io/blog/how-to-use-python-data-classes/) to model these parameters.

$\color{red}{\text{TODO: }}$ Implement `Camera` in `src/data_model.py`

In [3]:
# Define the parameters for Skydio VT300L - Wide camera
# Ref: https://support.skydio.com/hc/en-us/articles/20866347470491-Skydio-X10-camera-and-metadata-overview
fx = 4938.56
fy = 4936.49
cx = 4095.5
cy = 3071.5
sensor_size_x_mm = 13.107 # single pixel size * number of pixels in X dimension
sensor_size_y_mm = 9.830 # single pixel size * number of pixels in Y dimension
image_size_x = 8192
image_size_y = 6144

camera_x10 = Camera(fx, fy, cx, cy, sensor_size_x_mm, sensor_size_y_mm, image_size_x, image_size_y)

In [4]:
print(f"X10 camera model: {camera_x10}")

X10 camera model: Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor=13.107x9.830mm, resolution=8192x6144px)


# Week 3: Camera Operations

We plan to write utility functions to
- project a 3D world point to an image
- Compute image footprint on a surface
- Compute the Ground Sampling Distance

## Project 3D world points into the image


![Camera Projection](assets/image_projection.png)
Reference: [Robert Collins CSE483](https://www.cse.psu.edu/~rtc12/CSE486/lecture12.pdf)


Equations to implement:
$$ x = f_x \frac{X}{Z} $$
$$ y = f_y \frac{Y}{Z} $$
$$ u = x + c_x $$
$$ v = y + c_y $$

$\color{red}{\text{TODO: }}$ Implement function `project_world_point_to_image` in `src/camera_utils.py`

In [5]:
point_3d = np.array([25, -30, 50], dtype=np.float32)
expected_uv = np.array([6564.80, 109.60], dtype=np.float32)
uv = project_world_point_to_image(camera_x10, point_3d)

print(f"{point_3d} projected to {uv}")

assert np.allclose(uv, expected_uv, atol=1e-2)

[ 25. -30.  50.] projected to [6564.7803   109.60571]


## Compute Image Footprint on the surface

We have written code to *project* a 3D point into the image. The reverse operation is reprojection, where we take $(x, y)$ and compute the $(X, Y)$ for a given value of $Z$. Note that while going from 3D to 2D, the depth becomes ambiguous so we need the to specify the $Z$.

An image's footprint is the area on the surface which is captured by the image. We can take the two corners of the image and reproject them at a given distance to obtain the width and length of the image.

$\color{red}{\text{TODO: }}$ Implement function `compute_image_footprint_on_surface` in `src/camera_utils.py`

In [6]:
footprint_at_100m = compute_image_footprint_on_surface(camera_x10, 100)
expected_footprint_at_100m = np.array([165.88, 124.46], dtype=np.float32)

print(f"Footprint at 100m = {footprint_at_100m}")

assert np.allclose(footprint_at_100m, expected_footprint_at_100m, atol=1e-2)


Footprint at 100m = [165.87831271 124.46090238]


In [7]:
footprint_at_200m = compute_image_footprint_on_surface(camera_x10, 200)
expected_footprint_at_200m = expected_footprint_at_100m * 2

print(f"Footprint at 200m = {footprint_at_200m}")

assert np.allclose(footprint_at_200m, expected_footprint_at_200m, atol=1e-2)

Footprint at 200m = [331.75662541 248.92180476]


## Ground Sampling Distance

Ground sampling distance is the length of the ground (in m) captured by a single pixel. We have the image footpring (the dimensions of ground captured by the whole sensor, and the number of pixels along the horizontal and vertical dimension. Can we get GSD from these two quantities?

Note: Please return just one value of the GSD. Take the mininum of the values along the two axes.

In [8]:
gsd_at_100m = compute_ground_sampling_distance(camera_x10, 100)
expected_gsd_at_100m = 0.0202

print(f"GSD at 100m: {gsd_at_100m}")

assert np.allclose(gsd_at_100m, expected_gsd_at_100m, atol=1e-4)

GSD at 100m: 0.020248817469059807


## Bonus: Reprojection from 2D to 3D

If we have a 2d pixel location of a point along with the camera model, can we go back to 3D?
Do we need any additional information.


$\color{red}{\text{TODO: }}$ Implement function `reproject_image_point_to_world` in `src/camera_utils.py` and demonstrate it by running it in the notebook. Confirm that your reprojection + projection function are by projecting a 3d point to image, and reprojecting it back to 3D.

In [9]:
from src.camera_utils import reproject_image_point_to_world

original_3d_point = np.array([25, -30, 50], dtype=np.float32)
print(f"Original 3D point: {original_3d_point}")

projected_2d = project_world_point_to_image(camera_x10, original_3d_point)
print(f"Projected to 2D: {projected_2d}")

reprojected_3d = reproject_image_point_to_world(camera_x10, projected_2d, original_3d_point[2])
print(f"Reprojected back to 3D: {reprojected_3d}")

print(f"\nDifference between original and reprojected: {np.abs(original_3d_point - reprojected_3d)}")

Original 3D point: [ 25. -30.  50.]
Projected to 2D: [6564.7803   109.60571]
Reprojected back to 3D: [ 25.000004 -30.000002  50.      ]

Difference between original and reprojected: [3.8146973e-06 1.9073486e-06 0.0000000e+00]


### Understanding the Ambiguity in 2D to 3D Reprojection

To go from 2D back to 3D, we **need additional information** - specifically the depth (Z coordinate). This is because:

1. **The projection from 3D to 2D loses depth information** - multiple 3D points along the same ray from the camera center will project to the same 2D pixel.

2. **Without depth, the problem is under-constrained** - given just a 2D pixel coordinate (u, v), there are infinite possible 3D points that could have projected to that pixel.

3. **The depth provides the missing constraint** - once we know the Z distance, we can uniquely determine the X and Y coordinates.

Let's demonstrate this by showing how different depths give different 3D points for the same 2D pixel:

In [10]:
image_point = np.array([4000, 3000])
print(f"Image point: {image_point}")

for depth in [50, 100, 200, 500]:
    world_point = reproject_image_point_to_world(camera_x10, image_point, depth)
    print(f"At depth {depth}m: X={world_point[0]:.2f}, Y={world_point[1]:.2f}, Z={world_point[2]:.2f}")
    reprojected_2d = project_world_point_to_image(camera_x10, world_point)
    print(f"  Projects back to: [{reprojected_2d[0]:.1f}, {reprojected_2d[1]:.1f}]")

Image point: [4000 3000]
At depth 50m: X=-0.97, Y=-0.72, Z=50.00
  Projects back to: [4000.0, 3000.0]
At depth 100m: X=-1.93, Y=-1.45, Z=100.00
  Projects back to: [4000.0, 3000.0]
At depth 200m: X=-3.87, Y=-2.90, Z=200.00
  Projects back to: [4000.0, 3000.0]
At depth 500m: X=-9.67, Y=-7.24, Z=500.00
  Projects back to: [4000.0, 3000.0]


# Week 4: Compute Distance Between Photos

The overlap and sidelap are the ratio of the dimensions shared between two photos. We already know the footprint of a single image at a given distance. Can we convert the ratio into actual distances? And how does the distance on the surface relate to distance travelled by the camera?

$\color{red}{\text{TODO: }}$ Implement `compute_distance_between_images` in `src/plan_computation.py`



In [11]:
computed_distances = compute_distance_between_images(camera_x10, dataset_spec)
expected_distances = np.array([15.17, 11.38], dtype=np.float32)

print(f"Computed distance for X10 camera with nominal dataset specs: {computed_distances}")

assert np.allclose(computed_distances, expected_distances, atol=1e-2)

Computed distance for X10 camera with nominal dataset specs: [15.16791291 11.38070491]


$\color{red}{\text{TODO: }}$ define >=2 more specifications/camera parameters and check the computed distances. Does that align with your expections


In [12]:
print("Baseline dataset_spec:", dataset_spec)
baseline_distances = compute_distance_between_images(camera_x10, dataset_spec)
print("Baseline distances (m) [dx, dy]:", baseline_distances)

# Double height -> expect footprint and distances ~2x
spec_height = copy.copy(dataset_spec)
spec_height.height = dataset_spec.height * 2.0
dist_height = compute_distance_between_images(camera_x10, spec_height)
print("\nDouble height")
print(" distances:", dist_height, " ratio to baseline:", dist_height / baseline_distances)
print("Distance increase factor (should be ~2x):", dist_height / baseline_distances)

# Halve focal length -> expect x2 larger footprint -> distances increase
cam_half_fx = copy.copy(camera_x10)
cam_half_fx.fx *= 0.5
cam_half_fx.fy *= 0.5
dist_half_fx = compute_distance_between_images(cam_half_fx, dataset_spec)
print("\nHalve focal length (fx,fy * 0.5)")
print(" distances:", dist_half_fx, " ratio to baseline:", dist_half_fx / baseline_distances)
print("Distance increase factor (should be ~2x):", dist_half_fx / baseline_distances)

# Halve image resolution -> expect smaller footprint -> distances decrease
cam_half_res = copy.copy(camera_x10)
cam_half_res.image_size_x //= 2
cam_half_res.image_size_y //= 2
dist_half_res = compute_distance_between_images(cam_half_res, dataset_spec)
print("\nHalf image resolution (pixels)")
print(" distances:", dist_half_res, " ratio to baseline:", dist_half_res / baseline_distances)
print("Distance decrease factor (should be ~0.5x):", dist_half_res / baseline_distances)

Baseline dataset_spec: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48m, scan_area=150x150m, exposure=2ms)
Baseline distances (m) [dx, dy]: [15.16791291 11.38070491]

Double height
 distances: [30.33582583 22.76140983]  ratio to baseline: [2. 2.]
Distance increase factor (should be ~2x): [2. 2.]

Halve focal length (fx,fy * 0.5)
 distances: [30.33582583 22.76140983]  ratio to baseline: [2. 2.]
Distance increase factor (should be ~2x): [2. 2.]

Half image resolution (pixels)
 distances: [7.58395646 5.69035246]  ratio to baseline: [0.5 0.5]
Distance decrease factor (should be ~0.5x): [0.5 0.5]


## Bonus: Non-Nadir photos

We have solved for the distance assuming that the camera is facing straight down to the ground. This is called [Nadir scanning](https://support.esri.com/en-us/gis-dictionary/nadir). However, in practise we might want a custom gimbal angle.

Your bonus task is to make the distance computation general. Introduce a double `camera_angle` parameter (which is the angle from the X-axis) in the dataset specification, and work out how to adapt your computation. Feel free to reach out to Ayush to discuss ideas and assumptions!

![Non Nadir Footprint](assets/non_nadir_gimbal_angle.png)

In [13]:
from src.plan_computation import compute_non_nadir_footprint, compute_distance_between_images_non_nadir

camera_angle_deg = 30
camera_angle_rad = np.deg2rad(camera_angle_deg)

# Compute for 30 degree tilt
footprint_non_nadir = compute_non_nadir_footprint(camera_x10, dataset_spec.height, camera_angle_rad)
print(f"Footprint at {dataset_spec.height}m with {camera_angle_deg}° tilt: {footprint_non_nadir}")

# Compute the distance between images using the new footprint
distances_non_nadir = compute_distance_between_images_non_nadir(camera_x10, dataset_spec, camera_angle_rad)
print(f"Distance between images (non-nadir, {camera_angle_deg}° tilt): {distances_non_nadir}")

Footprint at 30.48m with 30° tilt: [50.55970971 43.80435364]
Distance between images (non-nadir, 30° tilt): [15.16791291 13.14130609]


# Week 5: Compute Maximum Speed For Blur Free Photos

To restrict motion blur due to camera movement to tolerable limits, we need to restrict the speed such that the image contents move less than 1px away. 

How much does 1px of movement translate to movement of the scene on the ground? It is the ground sampling distance!
From previous week, we know that this is the maximum movement the camera can have. 
We have the distance now. To get speed we need to divide it with time. Do we have time already in our data models?

$\color{red}{\text{TODO: }}$ Implement `compute_speed_during_photo_capture` in `src/plan_computation.py`.

In [ ]:
computed_speed = compute_speed_during_photo_capture(camera_x10, dataset_spec, allowed_movement_px=1)
expected_speed = 3.09

print(f"Computed speed during photo captures: {computed_speed:.2f}")

assert np.allclose(computed_speed, expected_speed, atol=1e-2)

$\color{red}{\text{TODO: }}$ define >= 2more specifications/camera parameters and check the computed distances. Does that align with your expectations


In [ ]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print(f"Computed distance: {computed_speed_:.2f}")

# Week 6: Generate Full Flight Plans  

We now have all the tools to generate the full flight plan.

Steps for this week:
1. Define the `Waypoint` data model. What attributes should the data model have?
   1. For Nadir scans, just the position of the camera is enough as we will always look drown to the ground.
   2. For general case (bonus), we also need to define where the drone will look at.
3. Implement the function `generate_photo_plan_on_grid` to generate the full plan.
   1. Compute the maximum distance between two images, horizontally and vertically.
   2. Layer the images such that we cover the whole scan area. Note that you need to take care when the scan dimension is not a multiple of distance between images. Example: to cover 45m length with 10m between images, we would need 4.5 images. Not possible. 4 images would not satisfy the overlap, so we should go with 5. How should we arrange 5 images in the given 45m.
   3. Assign the speed to each waypoint.

$\color{red}{\text{TODO: }}$ Implement:
- `Waypoint` in `src/data_model.py`
- `generate_photo_plan_on_grid` in `src/plan_computation.py`.

In [ ]:
computed_plan = generate_photo_plan_on_grid(camera_x10, dataset_spec) 

print(f"Computed plan with {len(computed_plan)} waypoints")

In [ ]:
MAX_NUM_WAYPOINTS_TO_PRINT = 20

for idx, waypoint in enumerate(computed_plan[:MAX_NUM_WAYPOINTS_TO_PRINT]):
    print(f"Idx {idx}: {waypoint}")
if len(computed_plan) >= MAX_NUM_WAYPOINTS_TO_PRINT:
    print("...")

## Bonus: Time computation 

if you have some time, you can implement a time computation function. We can make the drone fly as fast as possible between photos, but make sure it can decelerate back to the required speed at the photos. Please use the following data: 
- Max drone speed: 16m/s.
- Max acceleration: 3.5 m/s^2.

Hint: you might need to use a trapezoidal/triangular speed profile

# Week 7: Visualize Flight Plans

This week, we will use a third party plotting framework called [Plotly](https://plotly.com/python/) to visualize our plans. Please follow this [tutorial](https://www.kaggle.com/code/kanncaa1/plotly-tutorial-for-beginners) to gain some basic experience with Plotly, and then come up with your own visualization function. You are free to choose to come up with your own visualization, and use something other than Plotly.

$\color{red}{\text{TODO: }}$ Implement `plot_photo_plan` in `src/visualization.py`

In [ ]:
fig = plot_photo_plan(computed_plan)
fig.show()

$\color{red}{\text{TODO: }}$ Perform the following experiments (and any other you can think of) where we change just one parameter of the input camera/dataset specification and observe the change in the output plan. 

1. Change overlap and confirm it affects the consecutive images
2. Change sidelap and confirm it does not affect the consecutive images
3. Change the height of the scan and document the affect on scan plans
4. Change exposure time

Each experiment should specify: 
1. Input params you are changing
2. Impact you observe
3. explanation behind the change in output (intuition based or a text explanation is preffered over using equations)
4. Practical implication of the correlation: how can I drone pilot use this result

In [ ]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.exposure_time_ms = 1000

print(camera_, dataset_spec_)

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()